**Model Config**
Use Encoder-Decoder model

In [1]:
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import pandas as pd
import json
import time
import math
import random

import data_module as dm

from sklearn.model_selection import train_test_split

SOS = 0
EOS = 1

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


**Data Preprocessing**

- Load CSV file dataset
- Create Torch Dataset
- Create Torch DataLoader
- Create padding func (with insertion of SOS and EOS tokens



In [2]:
train_dl, test_dl, train_len, test_len = dm.get_loader(use_embedding = True)

In [3]:
it = iter(train_dl)
fs = next(it)
tt = fs[2]
print(tt)

tensor([ 540.,  315.,  510.,  270.,  285.,  885.,  555.,  555.,  375.,  885.,
         675.,  930., 1350.,  405.,  825.,  390.,  720.,  645.,  555.,  465.,
         255.,  540.,  795.,  780.,  855.,  540.,  315.,  180., 1020.,  345.,
         690.,  540.,  270.,  615., 1170.,  885., 1065.,  435., 5100.,  345.,
         465., 1695.,  870.,  315.,  795.,  660.,  285., 1140.,  675.,  480.,
         270.,  780., 2385.,  525., 1005.,  660.,  855., 1095., 1470.,  405.,
         450.,  810., 1530.,  360.])


**Model Building**
- Encoder-Decoder architecture
    - Encoder -> MLP or CNN
    - Decoder -> LSTM RNN
    - Batch Normalization in both


In [26]:
""" Model Architectures """

class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        
        self.fc1 = nn.Linear(594, 512)
        self.bn1 = nn.BatchNorm1d(512)
        self.fc2 = nn.Linear(512, 128)
        
        self.dropout = nn.Dropout(0.4)
    
    def forward(self, input):
        x = self.dropout(F.relu(self.bn1(self.fc1(input.float()))))
        x = F.relu(self.fc2(x))
        x = x.view(1, -1, 128)
        
        return x

In [141]:
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        
        self.emb = nn.Embedding(len(list(dm.coord_2_idx.keys())), 1024)
        self.gru = nn.GRU(1024, 128, num_layers=2)
        self.softmax = nn.Softmax(dim=-1)
        
    def forward(self, in_feats, hidden):
        embedded = self.emb(in_feats.to(torch.int64)).to(device)
        embedded = F.relu(embedded.squeeze(2))
        do, hidden = self.gru(embedded, hidden.to(torch.float))
        do = self.softmax(do)
        
        return do, hidden
    
    def init_hidden(self):
        return torch.zeros((2, 64, 128))

In [142]:
class Pred(nn.Module):
    def __init__(self):
        super(Pred, self).__init__()
        
        self.fc1 = nn.Linear(256+594, 256) # Take in 128 feats from gru hidden plus categoricals
        self.bn1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256,1)
        self.sm = nn.Sigmoid()

        self.dropout = nn.Dropout(0.4)
    def forward(self, in_feats):
        x = self.dropout(F.relu(self.bn1(self.fc1(in_feats))))
        x = self.fc2(x)
        x = self.sm(x) * 3 * 716.4264615618442
        return x

In [143]:
class RMSE(nn.Module):
    def __init__(self):
        super(RMSE, self).__init__()
        
        self.crit = nn.MSELoss()
    
    def forward(self, x, y):
        return torch.sqrt(self.crit(x.squeeze(0).to(torch.float64), y.squeeze(0).to(torch.float64)))

**Model Training**

In [144]:
# Trainer helper functions from 
# https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html#the-seq2seq-model
def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s' % (asMinutes(s))

In [145]:
def train(in_feats, cl, tt, encoder, decoder, pred, enc_optim, dec_optim, pred_optim, criterion):
    #enc_optim.zero_grad()
    dec_optim.zero_grad()
    pred_optim.zero_grad()
    
    in_feats = in_feats.to(device)
    cl = cl.to(device)
    tt = tt.to(device)
    
    loss = 0
    
    
    #hidden = encoder(in_feats)
    hidden = decoder.init_hidden()
    
    di = cl[0].unsqueeze(0).to(device)
    dh = hidden.to(device)
                    
    use_teacher_forcing = True if random.random() < 0 else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for i in range(len(cl) - 1):
            do, dh = decoder(di, dh)
            do = do.to(device)
            dh = dh.to(device)
            di = cl[i + 1].unsqueeze(0).to(device)  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for i in range(len(cl)):
            do, dh = decoder(di, dh)
            do = do.to(device)
            dh = dh.to(device)
            
            topv, topi = do.topk(1, dim=2)
            di = topi.detach().to(device)  # detach from history as input

    dh = dh.reshape((1, 64, -1))
    pred_in = torch.cat((in_feats, dh.squeeze(0)), dim=1).to(device)

    pred_time = pred(pred_in)
    tt = tt.unsqueeze(-1).to(torch.int64)

    loss += criterion(pred_time, tt)
    loss.backward()
        
    #enc_optim.step()
    dec_optim.step()
    pred_optim.step()
    
    return loss.item()

In [146]:
def trainEpochs(encoder, decoder, predictor, n_epochs, print_every=1000, eval_every = 5, learning_rate=0.003):
    start = time.time()
    print_loss_total = 0  # Reset every print_every

    enc_optim = optim.AdamW(encoder.parameters(), lr=learning_rate)
    dec_optim = optim.AdamW(decoder.parameters(), lr=learning_rate)
    pred_optim = optim.AdamW(predictor.parameters(), lr=learning_rate)
    criterion = RMSE()
    es = optim.lr_scheduler.ReduceLROnPlateau(enc_optim, 'min', 0.25, 3)
    ds = optim.lr_scheduler.ReduceLROnPlateau(dec_optim, 'min', 0.25, 3)
    ps = optim.lr_scheduler.ReduceLROnPlateau(pred_optim, 'min', 0.25, 3)

    
    epoch_loss_max = math.inf

    for epoch in range(n_epochs):
        for i, data in enumerate(train_dl): # change back to train_dl
            if i > 1:
                break
            in_feats = data[0]

            if (in_feats.shape[0] != 64):
                continue

            cl = data[1]

            tt = data[2]
            loss = train(in_feats, cl, tt, encoder, decoder, pred, enc_optim, dec_optim, pred_optim, criterion)
            print_loss_total += loss


            if (i % 1000 == 0):
                if (print_loss_total < epoch_loss_max):
                    epoch_loss_max = print_loss_total
                    torch.save({
                            'epoch': epoch,
                            'encoder_state_dict': encoder.state_dict(),
                            'encoder_optimizer_state_dict': enc_optim.state_dict(),
                            'decoder_state_dict': decoder.state_dict(),
                            'decoder_optimizer_state_dict': dec_optim.state_dict(),
                            'loss': print_loss_total,
                            }, 'model.pt')

            if (i % print_every == 0) and (i != 0): # Change back to i
                print_loss_avg = print_loss_total / print_every
                print_loss_total = 0



                print('Epoch: %d Elapsed: %s Percent of epoch Complete: (%d%%) %.4f' % (epoch, timeSince(start, i / (train_len / 128)),
                                                                  i / (train_len / 64) * 100, print_loss_avg))


            if (i % eval_every == 0) and (i != 0):
                print('*****EVALUATING*****')
                eval_loss = eval_epoch(encoder, decoder, pred, epoch)
                es.step(eval_loss)
                ds.step(eval_loss)
                ps.step(eval_loss)





In [147]:
def evaluate(encoder, decoder, pred, in_feats, cl, tt, it, max_len=1500):
    with torch.no_grad():
        
        eval_loss = RMSE()
        in_feats = in_feats.to(device)
        cl = cl.to(device)
        tt = tt.to(device)

        hidden = encoder(in_feats)


        di = torch.zeros((1,64,1)).to(device)
        dh = hidden.to(device)

        for i in range(150):
            do, dh = dec(di, dh)

            topv, topi = do.topk(1, dim=2)
            di = topi.detach().to(device)  # detach from history as input
            
        pred_in = torch.cat((in_feats.squeeze(0), dh.squeeze(0)), dim=1).to(device)
        pred_time = pred(pred_in)
        if (it < 1):
            print(pred_time)
            print(tt)
        l = eval_loss(pred_time, tt.unsqueeze(-1).to(torch.int64))
        
        return l
    
def eval_epoch(encoder, decoder, pred, epoch):
    encoder.eval()
    decoder.eval()
    pred.eval()

    accs = []
    for i, data in enumerate(test_dl):
        in_feats = data[0]
        if (in_feats.shape[0] != 64):
            continue
        cl = data[1]
        tt = data[2]
        accs.append(evaluate(encoder, decoder, pred, in_feats, cl, tt, i))
        
        if (i > 100):
            break
    
    epoch_acc = (sum(accs) / len(accs)) if len(accs) > 0 else 0
    print('Epoch: %d, Loss on test: %.4f' % (epoch, epoch_acc))
    encoder.train()
    decoder.train()
    pred.train()
    return epoch_acc

In [ ]:
enc = Encoder().to(device)
dec = Decoder().to(device)
pred = Pred().to(device)

trainEpochs(enc, dec, pred, 100, print_every=1, eval_every = 500, learning_rate = 0.003)


Epoch: 0 Elapsed: 0m 4s Percent of epoch Complete: (0%) 1310.2765
Epoch: 1 Elapsed: 0m 5s Percent of epoch Complete: (0%) 1446.5200
Epoch: 2 Elapsed: 0m 5s Percent of epoch Complete: (0%) 1198.0959
Epoch: 3 Elapsed: 0m 8s Percent of epoch Complete: (0%) 1372.3197
Epoch: 4 Elapsed: 0m 10s Percent of epoch Complete: (0%) 941.3321
Epoch: 5 Elapsed: 0m 11s Percent of epoch Complete: (0%) 1154.8363
Epoch: 6 Elapsed: 0m 12s Percent of epoch Complete: (0%) 1069.5322
Epoch: 7 Elapsed: 0m 15s Percent of epoch Complete: (0%) 1124.5506
Epoch: 8 Elapsed: 0m 17s Percent of epoch Complete: (0%) 1003.0706
Epoch: 9 Elapsed: 0m 18s Percent of epoch Complete: (0%) 1122.7804
Epoch: 10 Elapsed: 0m 19s Percent of epoch Complete: (0%) 1097.0497
Epoch: 11 Elapsed: 0m 22s Percent of epoch Complete: (0%) 918.7956
Epoch: 12 Elapsed: 0m 23s Percent of epoch Complete: (0%) 828.2552
Epoch: 13 Elapsed: 0m 24s Percent of epoch Complete: (0%) 1197.4323
Epoch: 14 Elapsed: 0m 25s Percent of epoch Complete: (0%) 1184.71

In [ ]:
def submission(enc, dec, pred):
    test_dl, test_len = dm.get_loader(test=True)
    trip_ids = []
    pred_times = []
    
    with torch.no_grad():
        enc.eval()
        dec.eval()
        pred.eval()
        for i, (trip_id, in_feats) in enumerate(test_dl):
            in_feats = in_feats.to(device)
            
            hidden = enc(in_feats)

            di = torch.Tensor([0]).unsqueeze(0).unsqueeze(0).to(device)
            dh = hidden.to(device)
                                
            for i in range(150):
                do, dh = dec(di, dh)
            
                topv, topi = do.topk(1, dim=2)
                di = topi.detach().to(device)  # detach from history as input
                if (topi == i):
                    break
                    
                if (i > 145):
                    print('reached 150')
            
            pred_in = torch.cat((in_feats, dh.squeeze(0)), dim=1).to(device)
            pred_idx = pred(pred_in)
            
            topv, topi = pred_idx.topk(1, dim=1)
            pred_time = dm.i2t[topi.item()]
            
            trip_ids.append(trip_id[0])
            pred_times.append(pred_time)

    df_sample = pd.read_csv("data/sampleSubmission.csv")
    df_sample["TRAVEL_TIME"] = pred_times
    df_sample.to_csv('submission.csv', index=None)

In [15]:
submission(enc, dec, pred)

NameError: name 'submission' is not defined